In [1]:
import random
import string
import time
import tensorflow as tf
import numpy as np
from scipy import sparse

In [2]:
alphabet = string.ascii_lowercase

In [3]:
#alphabet= 'abc'

In [4]:
test_min_str_length = 2
test_max_str_length = 25

In [6]:
list_of_strs = [''.join([alphabet[random.randrange(0, len(alphabet))] 
        for i in range(random.randrange(test_min_str_length, test_max_str_length))]) for j in range(100000)]

In [7]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [8]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [9]:
char_dict[' '] = max(char_dict.values())+1

In [10]:
max_length = max([len(i) for i in list_of_strs])

In [11]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix(
        [0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [12]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()

In [13]:
print ('time to generate one hot representation:%.3g s' % (end_time-start_time))

time to generate one hot representation:691 s


In [14]:
sparse_strs.shape

(100000, 648)

In [15]:
n_tiles = sparse_strs.shape[0]

In [16]:
max_length = sparse_strs.shape[1]

In [17]:
max_str_length = max([len(i) for i in list_of_strs])

In [18]:
vector_data = tf.sparse_placeholder(dtype=tf.float32, shape=sparse_strs.shape)

In [19]:
indices = np.reshape(np.concatenate(sparse_strs.nonzero()), [2,-1])

In [ ]:
#Instead of precomputing can we use a while loop?

In [20]:
#Tile reference sparse matrix (this is the b matrix)
ref_tiles = []
for k in range(sparse_strs.shape[0]):
    current_indices = np.concatenate([np.reshape(np.array([k for i in range(indices.shape[1])]), 
                                                 [1, indices.shape[1]]), indices])   
    if k==0:
        ref_tiles = np.transpose(current_indices)
    else:
        ref_tiles = np.concatenate([ref_tiles, np.transpose(current_indices)])

KeyboardInterrupt: 

In [ ]:
#Tile acting sparse matrix (a matrix)
act_tiles = []
for k in range(sparse_strs.shape[0]):

    str_indices = np.concatenate([sparse_strs.nonzero()[1][k*max_str_length:(k+1)*max_str_length] for i in range(sparse_strs.shape[0])]) 

    row_indices = np.concatenate([[i for k in range(max_str_length)] for i in range(sparse_strs.shape[0])]) 

    tiled_array = np.concatenate([np.reshape(row_indices, [row_indices.shape[0], 1]), 
                np.reshape(str_indices, [str_indices.shape[0], 1])], axis=1)
    
    current_indices=np.concatenate([np.reshape(np.array([k for i in range(indices.shape[1])]), [indices.shape[1], 1]),
                tiled_array], axis=1)
    
    if k==0:
        act_tiles = current_indices
    else:
        act_tiles = np.concatenate([act_tiles, current_indices])       

In [ ]:
assert ref_tiles.shape == act_tiles.shape

In [ ]:
tf.reset_default_graph()

In [ ]:
ref_sparse = tf.sparse_placeholder(dtype=tf.float32, shape=[sparse_strs.shape[0]]+ list(sparse_strs.shape))
act_sparse = tf.sparse_placeholder(dtype=tf.float32, shape=[sparse_strs.shape[0]]+ list(sparse_strs.shape))

In [ ]:
hamming_dists = tf.abs(tf.sparse_add(ref_sparse, act_sparse))

In [ ]:
hamming_dist_sum = tf.sparse_reduce_sum_sparse(hamming_dists, axis=2)

In [ ]:
#nearest_neighbors = tf.nn.top_k(hamming_dist_sum, k=2, sorted=True)

In [ ]:
start_time = time.time()
with tf.Session() as sess:
    t = sess.run(hamming_dist_sum, {ref_sparse.indices:ref_tiles, 
                               ref_sparse.values:[1 for i in range(ref_tiles.shape[0])],
                                           act_sparse.indices:act_tiles, 
                               act_sparse.values:[-1 for i in range(act_tiles.shape[0])]})
end_time = time.time()

In [ ]:
print('time to process:%.3g s' % (end_time - start_time))

In [88]:
rows = t.indices[:,0]
cols = t.indices[:,1]
data = t.values

In [89]:
test = sparse.csr_matrix((data, (rows, cols)), shape=t.dense_shape)

In [90]:
args = np.argsort(test.toarray(), axis=-1)

In [91]:
def map_nearest_neighbor(string_list, top_args, top_values):
    
    if type(string_list)!=np.array:
        string_list = np.array(string_list)
    
    results = []
    values = []
    for idx, top_arg in enumerate(top_args):
        results.append(string_list[top_arg])
        values.append(top_values[idx])
    return np.array(results), np.array(values)

In [92]:
top_args = args[:,:4]

In [93]:
top_values = test[:,:4].toarray()

In [94]:
nn_results = map_nearest_neighbor(list_of_strs, top_args, top_values)

In [95]:
nn_results[0]

array([['qeeg', 'lcrg', 'qshp', 'tce'],
       ['ocebyxaw', 'oyobsqaf', 'tce', 'cjephhaq'],
       ['rhsviuu', 'rjsw', 'rusz', 'obcviq'],
       ...,
       ['fvdngxfwehmwesc', 'whdeogawevjn', 'nlqnwoewztowevqh',
        'kvhmykfuet'],
       ['xjoianhlilxoxep', 'xampmnhyvtfvdae', 'uxtlarhlhfrws',
        'sahivntgiqiuodf'],
       ['iogqozpnigzqhbcp', 'qiubcnpzictvlbb', 'iottxipqv',
        'ivsqszokwsggcqni']], dtype='<U24')